In [ ]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 12
import seaborn as sns
sns.set_style('darkgrid')
import numpy as np
from time import time
from tqdm import tqdm
import pandas as pd
from collections import OrderedDict

from utils import generator, online_matchmaking, metrics, optimizers, actions, obj_functions, dyn_systems
from utils.base import Resource
import const_define as cd

## Generate data

In [ ]:
seed=424242
dataset = 'strong_unbalanced'

labelsAI = {i:f'AILabel{i+1}'for i in range(9)}

countries = {i+9:f'Country{i+1}'for i in range(5)}

r_rank, q_rank = None, None

# Balanced
if dataset == 'balanced':
    q_countries = (1/len(countries),) * len(countries)
    r_countries = q_countries
# Mild Unbalanced
elif dataset == 'mild_unbalanced':
    r_countries = (0.15, 0.15, 0.4, 0.15, 0.15)
    q_countries = (1/len(countries),) * len(countries)
# Strong Unbalanced
elif dataset == 'strong_unbalanced':
    r_countries = (0.05, 0.05, 0.8, 0.05, 0.05)
    q_countries = (1/len(countries),) * len(countries)
else:
    raise ValueError()

g = generator.StrwaiDataGenerator(labelsAI,
                        countries,
                        q_rank=q_rank,
                        r_rank=r_rank,
                        q_countries=q_countries,
                        r_countries=r_countries,
                        seed=seed)

#print(g)
n_resources = 40
n_queries = 100
queries_scores, resources_scores = g.generate_data(n_queries, n_resources)
g.plot_data(queries_scores, resources_scores, name=dataset, save=True)



## Test with Different Thresholds

In [ ]:
# Create Resources
resources_ids = list(range(len(resources_scores)))
resources_names = [f'res_{i}' for i in resources_ids]
resources_descr = [f'descr_{i}' for i in resources_ids]
resources_scores_dict = [{str(l):resources_scores[i,l] for l in range(g.n_labelsAI + g.n_countries)} for i in resources_ids]

resources_list = [Resource(name=resources_names[i], identifier=resources_ids[i], description=resources_descr[i],
                 scores=resources_scores_dict[i]) for i in resources_ids]
# Create Quieries
queries_scores_list = [{str(l):queries_scores[i,l] for l in range(g.n_labelsAI + g.n_countries)} for i in range(len(queries_scores))]
# Define matching algorithm
matching_metrics = online_matchmaking.CosineSimilarityScore()
mm_alg = online_matchmaking.OnlineMatchmaking(matching_metrics)
mm_alg.loadResources(resources_list)

# Listing protected label idxs
protected_labels = [9,10,11,12,13]
# Defining resources scores matrix
resources_scores = np.zeros((len(resources_list),g.n_labelsAI + g.n_countries))
for i in range(len(resources_list)):
    resources_scores[i] = np.array([resources_list[i].scores[str(j)] for j in range(g.n_labelsAI + g.n_countries)])

In [ ]:
records = {}
path = os.path.join(cd.PROJECT_DIR,'records', f'{dataset}/')
imgpath = os.path.join(cd.PROJECT_DIR,'images', f'{dataset}/')
records_list = ['true_rank',  # true rank (unmodified output of the matchmaking alg)
                'actioned_rank', # rank resulting from the application of the the actions found in the previous search to the true rank
                'provided_metrics',  # metrics of the actioned rank (which is the one provided to the user)
                'approx_metrics', # metrics of the approximated rank (it's obtained by applying the actions found in the current search to the actioned rank)
                'state', # record of the dynamical system evolution for each query (it's equal to the metrics itself if no search was performed)
                'cost_fn']  # approximation error (it's equal to -1 if no search was performed)

for file in records_list:
    fname = os.path.join(path,file + '.pkl')
    # Load existing records (if any)
    if os.path.isfile(fname):
        records[file] = pd.read_pickle(fname)
        print(f'Records loaded for {file}, with {len(records[file].columns)} experiments')
    else:
        print(f'No records found for {file} in {path}')
        records[file] = {}

In [ ]:

# Define desired dynamical behaviour
A = np.array([[0.5, 0.],
              [0., 0.5]])
# Fairness thresholds
# DIDI and rank distance
mu_dict = {0:np.array([1.,1.]),
           1:np.array([0., 1.]),
           2:np.array([1., 0.]),
          3:np.array([0.5, 0.2]),
          4:np.array([0.4,0.2]), 
          5:np.array([0.3,0.2]),
          6:np.array([0.1,0.1])} 

previous_queries_method = ['actionable'] 

for mu in mu_dict:
    
    if mu in records['true_rank'].keys():
        print('Already existing experiment')
        continue
    else:
        print(f'Test framework with threshold conf. No. {mu}')

        dyn_sys=dyn_systems.DynSystemMin(A,mu)
        for file in records_list:
            records[file][mu] = {k:None for k in previous_queries_method}
            
        for mode in previous_queries_method:

            cd.set_seed(seed)
            # Creating DIDI metrics object
            didi_obj = metrics.DIDI(seed,resources_scores, protected_labels, impact_function=metrics.power_law)

            # Define the metrics
            metrics_dict = OrderedDict(DIDI=didi_obj,
                           rank_dist=metrics.rank_similarity)
            metrics_obj = metrics.Metrics(metrics_dict, seed=seed)
            args_metrics = {k:{} for k in metrics_obj.metrics_dict}

            # Define objective function
            obj_fn = obj_functions.obj_fn_l2_wHistory
            # Define args for obj function
            metrics_weight = {'DIDI':1.,
                           'rank_dist':1.}

            obj_fn_args = {'resources_list':resources_list, 
                           'metrics':metrics_obj,
                          'args_metrics':args_metrics,
                          'metrics_weight':metrics_weight}


            # Define searching algorithm
            max_iter = 200
            patience = 20
            tolerance = 1e-4
            rnd_search = optimizers.RandomWalk_wHistory(max_iter=max_iter,
                                                n_samples=10,
                                                decay_rate=0.04,
                                                cost_fn=obj_fn,
                                                 patience=patience,
                                                 tolerance=tolerance,
                                                mode=mode,
                                                seed=seed,
                                              verbose=False)
            
            for file in records_list:
                if file in ['provided_metrics', 'approx_metrics']:
                    records[file][mu][mode] = {name:[] for name in metrics_obj.metrics_dict}
                else:
                    records[file][mu][mode] = []

            # Number of previous ranks to consider
            K = 5 
            counter = 0
            # We assume to not modify the very first query rank
            previous_flagged_resources = np.zeros((len(resources_list),),)

            # Loop over the queries
            for q in tqdm(queries_scores_list):
                # 1) Compute original rank r of Resources resulting from q
                true_rank = OrderedDict(mm_alg.matchOne(query_scores=q))
                records['true_rank'][mu][mode].append(true_rank)
                # 2) Perform actions
                # Change flag of resources
                resources_list_flagged = [Resource(name=resources_list[i].name, identifier=resources_list[i].identifier,
                                                        description=resources_list[i].description,
                                                        scores=resources_list[i].scores, at_bottom_flag=previous_flagged_resources[i]) for i in
                                          range(len(previous_flagged_resources))]
                # Put at the bottom
                actioned_rank, _ = actions.at_bottom(true_rank, resources_list_flagged)
                records['actioned_rank'][mu][mode].append(actioned_rank)
                # 3) Compute metrics with rank and actioned rank
                m = np.zeros((metrics_obj.n_metrics,))
                for i,name in enumerate(metrics_obj.metrics_dict):
                    m[i] = metrics_obj.metrics_dict[name](actioned_rank, true_rank, **args_metrics[name])
                    records['provided_metrics'][mu][mode][name].append(m[i])

                # 4) If metrics > \mu, select resources to put at the bottom
                if not np.all(m <=mu_dict[mu]):
                    # 5) Random search over \theta to minimize 
                    #the distance between the metrics with the new rank and the expected behaviour A

                    # Compute dynamical state to approximate
                    x = dyn_sys(m)
                    records['state'][mu][mode].append(x)
                    # Define starting point for search
                    w0 = np.random.choice(a=[0, 1], size=(len(resources_list),), p=[0.5, 0.5])


                    # Number of queries to be considered
                    if counter < K:
                        n_ranks=counter
                    else:
                        n_ranks=K

                    # Compute state for previous queries results 
                    previous_metrics = [np.array(records['provided_metrics'][mu][mode][name][:n_ranks])[:,None] for name in metrics_obj.metrics_dict]
                    previous_metrics = np.concatenate(previous_metrics, axis=1)
                    x_previous = np.array([dyn_sys(prev_m) for prev_m in previous_metrics])
                    # Define args for obj function
                    obj_fn_args['x']=x,
                    obj_fn_args['rank']=actioned_rank
                    obj_fn_args['x_previous'] = x_previous
                    # Searching
                    flag_history, cost_history, flipping_probs, k = rnd_search(flag=w0, 
                                                                                 previous_ranks=records['actioned_rank'][mu][mode][:n_ranks], 
                                                                                 args=obj_fn_args)

                    # Update set of actions
                    previous_flagged_resources = flag_history[-1]
                    # Store results
                    cost = cost_history[-1][0]
                    x_approx = cost_history[-1][1]
                    for i, name in enumerate(metrics_obj.metrics_dict):
                        records['approx_metrics'][mu][mode][name].append(x_approx[i]) 
                    records['cost_fn'][mu][mode].append(cost)
                else:
                    # There has been no approximation process, thus no update is needed for the actions
                    # and the approx metrics is exaclty the real one
                    for i, name in enumerate(metrics_obj.metrics_dict):
                        records['approx_metrics'][mu][mode][name].append(m[i]) 
                    records['state'][mu][mode].append(m)
                    records['cost_fn'][mu][mode].append(-1)
                counter+=1
# To dict
for file in records_list:
    records[file] = pd.DataFrame(records[file])

In [ ]:
if not os.path.isdir(path):
    os.makedirs(path)
    
for file in ['true_rank','actioned_rank','provided_metrics','approx_metrics','state','cost_fn']:
    fname = os.path.join(path,file + '.pkl')
    records[file].to_pickle(fname)
    print(f'Saved records for {file}')

## Comparison Plots

In [ ]:
records = {'balanced':{},
          'mild_unbalanced':{},
          'strong_unbalanced':{}}
mu_dict = {0:np.array([1.,1.]),
           1:np.array([0., 1.]),
           2:np.array([1., 0.]),
          3:np.array([0.5, 0.2]),
          4:np.array([0.4,0.2]), 
          5:np.array([0.3,0.2]),
          6:np.array([0.1,0.1])} 

for dataset in records.keys():
    print(f'Dataset: {dataset}')
    path = os.path.join(cd.PROJECT_DIR,'records', f'{dataset}/')
    records_list = ['true_rank',  # true rank (unmodified output of the matchmaking alg)
                    'actioned_rank', # rank resulting from the application of the the actions found in the previous search to the true rank
                    'provided_metrics',  # metrics of the actioned rank (which is the one provided to the user)
                    'approx_metrics', # metrics of the approximated rank (it's obtained by applying the actions found in the current search to the actioned rank)
                    'state', # record of the dynamical system evolution for each query (it's equal to the metrics itself if no search was performed)
                    'cost_fn']  # approximation error (it's equal to -1 if no search was performed)

    for file in records_list:
        fname = os.path.join(path,file + '.pkl')
        # Load existing records (if any)
        if os.path.isfile(fname):
            records[dataset][file] = pd.read_pickle(fname)
            print(f'Records loaded for {file}, with {len(records[dataset][file].columns)} experiments')
        else:
            print(f'No records found for {file} in {path}')
            records[dataset][file] = {}

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 19
import seaborn as sns
sns.set_style('whitegrid')

imgpath = os.path.join(cd.PROJECT_DIR,'images', f'comparison/')

if not os.path.isdir(imgpath):
    os.makedirs(imgpath)
    
metrics_names= ['DIDI',
               'rank_dist']

colors = {'balanced':'green',
          'mild_unbalanced':'#b100e9',
          'strong_unbalanced':'red'}

lighter_colors = {'balanced':'#52be80',
          'mild_unbalanced':'#cb75e7',
          'strong_unbalanced':'#ff6f6f'}

markers = {'balanced':'.',
          'mild_unbalanced':'x',
          'strong_unbalanced':'^'}

for mu in mu_dict:
    print('*'*40)
    print('EXP.',mu)
    print('*'*40)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,6), dpi=130)
    fname = f'exp_{mu}.eps'
    # Loop on metrics
    for k, name in enumerate(metrics_names):
        if name=='rank_dist':
            name_ = 'Rank Distance'
        else:
            name_=name
        # Loop on dataset
        for dataset in ['balanced','mild_unbalanced','strong_unbalanced']:
            if dataset=='mild_unbalanced':
                dt = 'Mild Unbalanced'
            elif dataset=='strong_unbalanced':
                dt = 'Strong Unbalanced'
            elif dataset=='balanced':
                dt = 'Balanced'
            rec = records[dataset]
            n_sample = len(rec['provided_metrics'][mu]['actionable'][name])
            ma_actionable =  pd.Series(rec['provided_metrics'][mu]['actionable'][name])
            print(dataset, name, np.mean(rec['provided_metrics'][mu]['actionable'][name]))
            axes[k].plot(range(n_sample), 
                     ma_actionable,
                     marker=markers[dataset],
                     linestyle='dotted',
                     color=lighter_colors[dataset], 
                     label=f'{dt}')
            
            if name == 'rank_dist':
                axes[k].set_ylim(-0.05,.4)
            else:
                axes[k].set_ylim(-0.05,1)


            axes[k].axhline(y=np.mean(rec['provided_metrics'][mu]['actionable'][name]),
                        color=colors[dataset], 
                        linestyle='-.',
                        linewidth=2.5,
                        label=f'{dt} (mean)')
        if mu != 0:
            axes[k].axhline(y=mu_dict[mu][k], color='blue', linewidth=2.5, linestyle='-', label='Threshold')
        else:
            axes[k].axhline(y=2., color='blue', linewidth=2.5, linestyle='-', label='Threshold')
        axes[k].set_xlabel('Queries')
        axes[k].set_ylabel(f'{name_}')
        if name == 'rank_dist' and mu==0:
            legend = axes[k].legend(loc='upper right',frameon=1, shadow=True)
            legend.get_frame().set_facecolor('white')
        elif name == 'DIDI' and mu==3:
            legend = axes[k].legend(bbox_to_anchor=(1,1.3),frameon=1, shadow=True)
            legend.get_frame().set_facecolor('white')
    plt.savefig(os.path.join(imgpath, fname), format='eps', bbox_inches="tight")
    plt.show()
plt.close('all')